In [ ]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory
'''
import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

'''

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

In [ ]:
import matplotlib.pyplot as plt
import os
import cv2
import random

In [ ]:
from keras.models import Model, Sequential
from keras.layers import Conv2D, MaxPooling2D, Flatten, Dense, Dropout, UpSampling2D
from keras.utils import np_utils

In [ ]:
def split (dataset, rows, columns):
    
    split_datasets = [ [] for i in range( rows * columns ) ]
    
    rows = int (96 / rows)
    columns = int (96 / columns)
    
    for data in dataset:
        data = data.reshape( 96//rows, rows,-1, columns).swapaxes(1,2).reshape(-1, rows, columns)
        
        for i in range(len(split_datasets)):
            split_datasets[i].append(data[i])
    
    return np.array(split_datasets)

In [ ]:
def combine(predictions):
    p = []
    for j in range(len(predictions[0])):
        y = np.array([np.argmax(predictions[i][j]) for i in range(len(predictions))])
        p.append(np.bincount(y).argmax())
    return np_utils.to_categorical(p)

In [ ]:
def one_hot_encode (label):
    label = np_utils.to_categorical(label)
    return label

In [ ]:
def display(image, rows, columns):

    fig = plt.figure(figsize = (5,5))

    for i in range( rows * columns):

        fig.add_subplot(rows, columns, i + 1)

        plt.imshow(image[i], cmap = "gray")
        plt.axis('off')
    plt.show()

In [ ]:
TRAINDATADIR = "../input/intel-image-classification/seg_train/seg_train"
TESTDATADIR = "../input/intel-image-classification/seg_test/seg_test"
CATEGORIES = ['buildings','forest','glacier','mountain','sea','street']

In [ ]:
IMG_SIZE = 96

In [ ]:
def create_data(DATADIR, CATEGORIES):
    data = []
    for category in CATEGORIES:
        path = os.path.join(DATADIR, category)
        class_num = CATEGORIES.index(category)
        for img in os.listdir(path):
            img_array = cv2.imread(os.path.join(path, img), cv2.IMREAD_GRAYSCALE)
            res_array = cv2.resize(img_array, (IMG_SIZE, IMG_SIZE))
            data.append([res_array, class_num])
    return data

In [ ]:
train_data = create_data(TRAINDATADIR, CATEGORIES)
print(len(train_data))

test_data = create_data(TESTDATADIR, CATEGORIES)
print(len(test_data))

In [ ]:
random.shuffle(train_data)
random.shuffle(test_data)

In [ ]:
train_images = []
label_train = []
for sample in train_data:
    train_images.append(np.array(sample[0]))
    label_train.append(sample[1])

test_images = []
label_test = []
for sample in test_data:
    test_images.append(np.array(sample[0]))
    label_test.append(sample[1])

In [ ]:
labels_train = one_hot_encode(label_train)
labels_test = one_hot_encode(label_test)

In [ ]:
train_images1 = np.asarray(train_images)
train_images1[0].shape

In [ ]:
test_images1 = np.asarray(test_images)
test_images1.shape

## SHL

In [ ]:
class MHL:
    def __init__(self, n_classes):
        self.model = Sequential()
        self.model.add(Flatten())
        self.model.add(Dense(64, activation = 'relu'))
        self.model.add(Dense(n_classes, activation = 'softmax'))
        self.model.compile(loss = 'categorical_crossentropy', optimizer = 'adam', metrics = 'accuracy')

In [ ]:
def create_model(x_images, y_images, x_labels, y_labels, r, c):
    mhl_rxc = []
    for i in range(r*c):
        mhl_rxc.append(MHL(6))

    for i in range(r*c):
        mhl_rxc[i].model.fit(x_images[i], x_labels, epochs = 10, batch_size = 128, verbose = 2)
        
    mhl_rxc[0].model.summary()
        
    mhl_rxc_pred = [ [] for i in range(r * c)]
    for i in range(r*c):
        mhl_rxc_pred[i] = mhl_rxc[i].model.predict(y_images[i])
        
    mhl_rxc_comb = combine(mhl_rxc_pred)
    
    from keras.metrics import Accuracy
    metric = Accuracy()
    metric.update_state(y_labels,mhl_rxc_comb)
    acc = metric.result().numpy()
    print(acc)
    return acc

## SHL 1x1 

In [ ]:
mhl1x1 = MHL(6)

mhl1x1.model.fit(train_images1, labels_train, validation_data = (test_images1, labels_test), epochs = 10, batch_size = 128, verbose = 2)

In [ ]:
mhl1x1.model.summary() #v1 SHL: 4068

In [ ]:
shl1x1 = MHL(6)

shl1x1.model.fit(train_images1, labels_train, validation_data = (test_images1, labels_test), epochs = 10, batch_size = 128, verbose = 2)

In [ ]:
shl1x1.model.summary() #v1 SHL: 2304

## SHL 2x2

In [ ]:
train = split(train_images1, 2, 2)
test = split(test_images1, 2, 2)

print(train[0][0].shape, test[0][0].shape)

In [ ]:
acc = create_model(train, test, labels_train, labels_test, 2, 2)
#v1 SHL: 1152

In [ ]:
print(acc)

## SHL 2x3

In [ ]:
train = split(train_images1, 2, 3)
test = split(test_images1, 2, 3)

print(train[0][0].shape, test[0][0].shape)

In [ ]:
acc = create_model(train, test, labels_train, labels_test, 2, 3)
#v1 shl2x3: 1152

In [ ]:
print(acc)

In [ ]:
acc = create_model(train, test, labels_train, labels_test, 2, 3)
#v1 shl2x3: 768

In [ ]:
print(acc)

## SHL 3x3

In [ ]:
train = split(train_images1, 3, 3)
test = split(test_images1, 3, 3)

print(train[0][0].shape, test[0][0].shape)

In [ ]:
acc = create_model(train, test, labels_train, labels_test, 3, 3)
#v1 shl2x3: 512

In [ ]:
print(acc)

## SHL 3x4

In [ ]:
train = split(train_images1, 3, 4)
test = split(test_images1, 3, 4)

print(train[0][0].shape, test[0][0].shape)

In [ ]:
acc = create_model(train, test, labels_train, labels_test, 3, 4)
#v1 shl2x3: 384

In [ ]:
print(acc)

## SHL 3x6

In [ ]:
train = split(train_images1, 3, 6)
test = split(test_images1, 3, 6)

print(train[0][0].shape, test[0][0].shape)

In [ ]:
acc = create_model(train, test, labels_train, labels_test, 3, 6)
#v1 shl3x4: 256

In [ ]:
print(acc)

## SHL 4x6

In [ ]:
train = split(train_images1, 4, 6)
test = split(test_images1, 4, 6)

print(train[0][0].shape, test[0][0].shape)

In [ ]:
acc = create_model(train, test, labels_train, labels_test, 4, 6)
#v1 shl4x6: 192

In [ ]:
print(acc)

## SHL 4x8

In [ ]:
train = split(train_images1, 4, 8)
test = split(test_images1, 4, 8)

print(train[0][0].shape, test[0][0].shape)

In [ ]:
acc = create_model(train, test, labels_train, labels_test, 4, 8)
#v1 shl4x8: 144

In [ ]:
print(acc)

## SHL 6x6

In [ ]:
train = split(train_images1, 6, 6)
test = split(test_images1, 6, 6)

print(train[0][0].shape, test[0][0].shape)

In [ ]:
acc = create_model(train, test, labels_train, labels_test, 6, 6)
#v1 shl6x6: 128

In [ ]:
print(acc)

## SHL 4x12

In [ ]:
train = split(train_images1, 3, 4)
test = split(test_images1, 3, 4)

print(train[0][0].shape, test[0][0].shape)

In [ ]:
acc = create_model(train, test, labels_train, labels_test, 3, 4)
#v1 shl

In [ ]:
print(acc)

## SHL 8x8

In [ ]:
train = split(train_images1, 3, 4)
test = split(test_images1, 3, 4)

print(train[0][0].shape, test[0][0].shape)

In [ ]:
acc = create_model(train, test, labels_train, labels_test, 6, 6)
#v1 shl6x6: 128

In [ ]:
print(acc)

## SHL 6x12

In [ ]:
train = split(train_images1, 6, 12)
test = split(test_images1, 6, 12)

print(train[0][0].shape, test[0][0].shape)

In [ ]:
acc = create_model(train, test, labels_train, labels_test, 6, 12)
#v1 shl6x12: 64

In [ ]:
print(acc)